In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import linregress
from scipy.stats import ttest_ind
import statsmodels.stats.power as smp

In [2]:
df_profiles=pd.read_csv("adjusted_data/profiles.csv")
df_processes=pd.read_csv("adjusted_data/processes.csv")
df_devices=pd.read_csv("adjusted_data/devices.csv")
df_connections = pd.read_csv("adjusted_data/connections.csv")
df_profiles.head()

,Unnamed: 0,company,imei,name,ssn,registration,username,user_id,mail,birthdate
0,0,Beck-Wright,863033069630348784,Gregory Santiago,493-70-5760,2023-05-04,zacharyclayton,474,xraymond@hotmail.com,1967-09-17 09:20:20.600858368
1,1,"Phillips, Dixon and Garrett",3590433799317661115,Tom Fisher,598-97-2312,2020-05-09,dustin49,1307,kevin90@hotmail.com,1967-09-17 09:20:20.600858368
2,2,Manning LLC,863033069630348826,Howard Garcia,383-26-6902,2021-08-21,davidstone,2255,holmestommy@yahoo.com,1967-09-17 09:20:20.600858368
3,3,"Duarte, Campbell and Escobar",863033069630348263,Andrew Montoya,556-81-7215,2023-03-12,juliehayes,1319,john38@gmail.com,1967-09-17 09:20:20.600858368
4,4,Mcclure-Lutz,863033069630348131,Kelly Holder,213-99-5419,2019-11-10,donna35,1883,rebecca75@gmail.com,1967-09-17 09:20:20.600858368
